<a href="https://colab.research.google.com/github/Dharani1999/Word-embedding-techniques/blob/master/Code_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import pickle
import gensim
from gensim import corpora, models, similarities
from gensim.models import Word2Vec, TfidfModel, LsiModel
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.fasttext import FastText
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
#!pip install git+https://github.com/huggingface/transformers
!pip install glove_python
from glove import Corpus, Glove# creating a corpus object

     |████████████████████████████████| 266kB 2.9MB/s 
  Created wheel for glove-python: filename=glove_python-0.1.0-cp36-cp36m-linux_x86_64.whl size=700234 sha256=cf96ac10f10f62b5ff0c7c49b4537111ddd266eb2a2664fc93dd96c41f93991c
  Stored in directory: /root/.cache/pip/wheels/88/4b/6d/10c0d2ad32c9d9d68beec9694a6f0b6e83ab1662a90a089a4b
Successfully built glove-python


In [3]:
def data_module(name, location, output_location, dict_location, max_seq_length=100):
  dataset1 = pd.read_csv(location)
  dataset = dataset1.iloc[0:1000,:]

  if name == 'word2vec':
    data_corpus, users_total = data_word2vec(dataset,max_seq_length)
  elif name == 'doc2vec':
    data_corpus, users_total = data_doc2vec(dataset,max_seq_length)
  elif name == 'lsi':
    data_corpus, users_total = data_lsi(dataset,max_seq_length,dict_loc=dict_location)
  elif name == 'tfidf':
    data_corpus, users_total = data_lsi(dataset,max_seq_length,dict_loc=dict_location)
  elif name == 'glove':
    data_corpus, users_total = data_word2vec(dataset,max_seq_length)
  elif name == 'hashing':
    data_corpus, users_total = data_hashing(dataset,max_seq_length)
  elif name == 'cooccur':
    data_corpus, users_total = data_hashing(dataset,max_seq_length)
  #elif name == 'fasttext':
   # data_corpus, users_total = data_word2vec(dataset,max_seq_length)
  #data_corpus.save('/content/drive/My Drive/Movielensdata/ml25m/data_corpus')
  #print(data_corpus)
  save_object(obj=data_corpus, filename=output_location)
  #return data_corpus, users_total

def data_word2vec(dataset,max_seq_length):
  dataset.sort_values(by=['userId','timestamp'],inplace=True)
  user_total = len(dataset['userId'].unique())
  
  #Selecting the most recent movies rated by each user and padding if necessary
  movie_list = []
  for i in range(user_total):
    list1 = []
    list1 = dataset.loc[dataset['userId'] ==(i+1),['movieId']]['movieId'].tolist()
    if len(list1)>max_seq_length:
      list1 = list1[(len(list1)-max_seq_length):]
    elif len(list1)<max_seq_length:
      list1 = list1+[0 for j in range((max_seq_length-len(list1)))]
      #for j in range((max_seq_length-len(list1))):
       # list1.append(0)
    movie_list.append(list1)
  
  #Selecting the most recent ratings rated by each user and padding if necessary
  rating_list =[]
  for i in range(user_total):
    list2 = []
    list2 = dataset.loc[dataset['userId'] ==(i+1),['rating']]['rating'].tolist()
    if len(list2)>max_seq_length:
      list2 = list2[(len(list2)-max_seq_length):]
    elif len(list2)<max_seq_length:
      list2 = list2+[0 for j in range((max_seq_length-len(list2)))]
      #for j in range((max_seq_length-len(list2))):
       # list2.append(0)
    rating_list.append(list2)
  
  #Creating user_id level transpose matrices
  movies_transpose = pd.DataFrame(data=movie_list,index=[i+1 for i in range(user_total)])
  movies_transpose.index.names = ['userId']
  #print(movies_transpose)

  ratings_transpose = pd.DataFrame(data=rating_list,index=[i+1 for i in range(user_total)])
  ratings_transpose.index.names = ['userId']
  #print(ratings_transpose)

  # Select features from original dataset to form a new dataframe 
  df1 = movies_transpose.iloc[:]# For each row, combine all the columns into one column
  df2 = df1.apply(lambda x: ','.join(x.astype(str)), axis=1)# Store them in a pandas dataframe
  df_clean = pd.DataFrame({'clean': df2})# Create the list of list format of the custom corpus for gensim modeling 
  sent = [row.split(',') for row in df_clean['clean']]

  return sent, user_total

def data_doc2vec(dataset,max_seq_length):
  Sent, user_total = data_word2vec(dataset,max_seq_length)
  tagged_data = []
  tags = []
  
  for i in range(user_total):
    tagged_data = tagged_data + [TaggedDocument(words=Sent[i], tags=[str(i)])]

  return tagged_data, user_total

def data_lsi(dataset,max_seq_length,dict_loc):
  Sent, user_total = data_word2vec(dataset,max_seq_length)
  dictionary = corpora.Dictionary(Sent)
  #print(dictionary.token2id)
  corpus = [dictionary.doc2bow(text) for text in Sent]
  dictionary.save(dict_loc)
  #corpus = np.array([[(id, freq) for id, freq in cp] for cp in corp])
  #corpus = gensim.matutils.Dense2Corpus(np.array(Sent),documents_columns=False)

  return corpus, user_total

def data_hashing(dataset,max_seq_length):
  Sent, user_total = data_word2vec(dataset,max_seq_length)
  corpus = [str(str(doc)[1:-1]) for doc in Sent]
  return corpus, user_total

In [4]:
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

def load_object(filename):
    with open(filename, 'rb') as input:
        pickle_object = pickle.load(input)
    return  pickle_object

In [5]:
def embedding_model(name,Data_location,model_save_location,matrix_location, vector_dims=10,Sg=1,size_window=3,topics=10,mini_count=1,num_workers=3,max_num_epochs = 100,alpha = 0.025,min_alpha=0.00025,dm=1, maxi_features=None):
  Data = load_object(Data_location)
  #print(Data)
  if name == 'word2vec':
    word2vec(input_data=Data, save_loc=model_save_location, vec_dims=vector_dims, SG=Sg, size_of_window=size_window, minimum_count=mini_count, no_workers=num_workers)
    #voc = model1.wv
    #words = list(model1.wv.vocab)
    #vectors = model1[model1.wv.vocab]
  elif name == 'doc2vec':
    doc2vec(input_data=Data, save_loc=model_save_location, vec_dims=vector_dims, alpha_=alpha, size_of_window=size_window, no_workers=num_workers, max_epochs=max_num_epochs,min_alpha_=min_alpha, minimum_count=mini_count, dms=dm)
  elif name == 'lsi':
    lsi(input_data=Data, save_loc=model_save_location, total_topics=topics)
  elif name == 'tfidf':
    tfidf(input_data=Data,save_loc=model_save_location)
  elif name == 'glove':
    glove_model(input_data=Data, vec_dims=vector_dims, size_of_window=size_window, save_loc=model_save_location, num_epochs=max_num_epochs, alpha_=0.05, num_threads=4)
  elif name == 'hashing':
    hashing(input_data=Data, vec_dims=vector_dims, save_loc=model_save_location)
  elif name == 'cooccur':
    co_occur(input_data=Data, maximum_features=maxi_features, save_loc=model_save_location, matrix_loc = matrix_location)
  #elif name == 'fasttext':
   # fast_text(input_data=Data, save_loc=model_save_location, vec_dims=vector_dims, SG=Sg, size_of_window=size_window, minimum_count=mini_count, no_workers=num_workers, alpha_=0.025)

def word2vec(input_data,save_loc,vec_dims,SG,size_of_window,minimum_count,no_workers):
  model = Word2Vec(input_data,min_count=minimum_count,size= vec_dims,workers=no_workers, window =size_of_window, sg = SG)
  model.save(save_loc)

def doc2vec(input_data,save_loc,vec_dims,alpha_,size_of_window,min_alpha_,minimum_count,dms,no_workers,max_epochs):
  model = Doc2Vec(size=vec_dims,
                alpha=alpha_, 
                min_alpha=min_alpha_,
                window = size_of_window,
                min_count=minimum_count,
                dm =dms)
  model.build_vocab(input_data)

  for epoch in range(max_epochs):
    #print('iteration {0}'.format(epoch))
    model.train(input_data, total_examples=model.corpus_count, epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
  model.save(save_loc)

def lsi(input_data,save_loc,total_topics):
  model = models.LsiModel(corpus=input_data, num_topics=total_topics)
  index = similarities.MatrixSimilarity(model[input_data])
  lsi_data = model[input_data]
  lsi_topics = model.print_topics()
  #for topic in lsi_topics:
    #print(topic)
  model.save(save_loc)

def tfidf(input_data,save_loc):
  model = models.TfidfModel(corpus=input_data)
  tfidf_data = model[input_data]

  tfidf_token= np.zeros((len(tfidf_data), 350), dtype=np.float64)
  tfidf_vals= np.zeros((len(tfidf_data), 350), dtype=np.float64)
 
  for i in range(len(input_data)):
    for k in range(len(list(tfidf_data)[i])):
      tfidf_token[i][k]=(list(tfidf_data))[i][k][0]
      tfidf_vals[i][k]=(list(tfidf_data))[i][k][1]
  tfidf_list=list(tfidf_data)
  #print(list(tfidf_data))
  model.save(save_loc)

def glove_model(input_data,vec_dims,size_of_window,save_loc,alpha_=0.05,num_epochs=30, num_threads=4):
  #importing the glove library
  corpus = Corpus() #training the corpus to generate the co occurence matrix which is used in GloVe
  corpus.fit(input_data, window=size_of_window)#creating a Glove object which will use the matrix created in the above lines to create embeddings
  #We can set the learning rate as it uses Gradient Descent and number of components
  glove = Glove(no_components=vec_dims, learning_rate=alpha_) 
  glove.fit(corpus.matrix, epochs=num_epochs, no_threads=4, verbose=True)
  glove.add_dictionary(corpus.dictionary)
  glove.save(save_loc)

def fast_text(input_data,save_loc,vec_dims,SG,size_of_window,minimum_count,no_workers,alpha_=0.025):
  model = FastText(min_count=minimum_count, alpha=alpha_, size= vec_dims, workers=no_workers, window =size_of_window)
  model.build_vocab(input_data)
  model.train(input_data, epochs=model.epochs, total_examples=model.corpus_count, total_words=model.corpus_total_words)
  model.save(save_loc)

def hashing(input_data,vec_dims,save_loc):
  model = HashingVectorizer(n_features=vec_dims)
  model.transform(input_data)
  #vectors = model.toarray()
  #vocab = model.get_feature_names()
  save_object(obj=model, filename=save_loc)

def co_occur(input_data, save_loc, matrix_loc, maximum_features):
  model = CountVectorizer(ngram_range=(1,1),max_features=maximum_features, token_pattern= r"(?u)\b\w+\b")
  X = model.fit_transform(input_data)
  Xc = (X.T * X)
  Xc.setdiag(0)
  #cooccur = Xc.todense()
  names = model.get_feature_names() # This are the entity names (i.e. keywords)
  save_object(obj=names, filename='/content/drive/My Drive/Movielensdata/ml25m/cooccur/vocab')
  df = pd.DataFrame(data = Xc.toarray(), columns = names, index = names)
  save_object(obj=model, filename=save_loc)
  save_object(obj=df, filename=matrix_loc)


In [6]:
def scoring_module(name,model_loc,data,dict_loc,matrix_location):
  if name == 'word2vec':
    scores_list = score_word2vec(model_location=model_loc,input_data=data)
  elif name == 'doc2vec':
    scores_list = score_doc2vec(model_location=model_loc,input_data=data)
  elif name == 'lsi':
    scores_list = score_lsi(model_location=model_loc,input_data=data, dict_location=dict_loc)
  elif name == 'tfidf':
    scores_list = score_tfidf(model_location=model_loc,input_data=data, dict_location=dict_loc)
  elif name == 'glove':
    scores_list = score_glove(model_location=model_loc, input_data=data)
  elif name == 'hashing':
    scores_list = score_hashing(model_location=model_loc, input_data=data)
  elif name == 'cooccur':
    scores_list = score_cooccur(matrix_loc=matrix_location, input_data=data)
  #elif name == 'fasttext':
   # scores_list = score_fasttext(model_location=model_loc, input_data=data)
  
  return scores_list

def score_word2vec(model_location,input_data):
  model = Word2Vec.load(model_location)
  scored_data = [model.wv[text] for text in input_data]
  return scored_data

def score_doc2vec(model_location,input_data):
  model = Doc2Vec.load(model_location)
  scored_data = [model.infer_vector(text) for text in input_data]
  return scored_data

def score_glove(model_location,input_data):
  model = Glove.load(model_location)
  scored_data = [model.word_vectors[model.dictionary[text]] for doc in input_data for text in doc]
  #model.word_vectors[]
  return scored_data

def score_fasttext(model_location, input_data):
  model = FastText(model_location)
  scored_data = [model[text] for text in input_data]
  return scored_data

def score_lsi(model_location, input_data, dict_location):
  #model = LsiModel.load(model_location)
  dictionary = corpora.Dictionary.load(dict_location)
  scored_data = [model[dictionary.doc2bow(text)] for text in input_data]
  return scored_data

def score_tfidf(model_location, input_data, dict_location):
  #model = TfidfModel.load(model_location)
  dictionary = corpora.Dictionary.load(dict_location)
  scored_data = [model[dictionary.doc2bow(text)] for text in input_data]
  return scored_data

def score_hashing(model_location, input_data):
  corpus = [str(str(doc)[1:-1]) for doc in input_data]
  model = load_object(model_location)
  X = model.transform(corpus)
  Doc_Term_Matrix = pd.DataFrame(X.toarray())
  Doc_Term_Matrix
  return Doc_Term_Matrix

def score_cooccur(matrix_loc, input_data):
  cooccur_matrix = load_object(matrix_loc)
  #print(cooccur_matrix)
  scored_data = [cooccur_matrix.loc[text].tolist() for doc in input_data for text in doc]
  return scored_data

1) Name can be 'word2vec' or 'doc2vec' or 'lsi' or 'tfidf' or 'glove'.

2) location is the path to input .csv file

3) output_location is the path to save the processed data

4) dict_location is the path to dictionary of LSI or TF-IDF

Change name, output_location for all models and dict_location for LSI and TF-IDF

In [7]:
data_module(name='cooccur',max_seq_length=100,location='/content/drive/My Drive/Movielensdata/ml25m/ratings.csv',output_location='/content/drive/My Drive/Movielensdata/ml25m/cooccur/data', dict_location='/content/drive/My Drive/Movielensdata/ml25m/tfidf/dict')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Change name, Data_location and model_save_location for all models

In [8]:
embedding_model(name='cooccur',Data_location='/content/drive/My Drive/Movielensdata/ml25m/cooccur/data',model_save_location='/content/drive/My Drive/Movielensdata/ml25m/cooccur/co_occur', matrix_location='/content/drive/My Drive/Movielensdata/ml25m/cooccur/matrix', vector_dims=10,Sg=1,size_window=3,topics=10,mini_count=1,num_workers=3,max_num_epochs = 10,alpha = 0.025,min_alpha=0.00025,dm=1,maxi_features=None)

In [9]:
scoring_input = [['899', '2161', '3949', '5878', '1175', '1237', '8154', '2843', '7365', '4422', '6016'],['1080', '3114', '3671', '2791', '1288', '1', '541', '2692', '7323', '8014', '6370', '4703', '5147']]

Change name, model_loc or all models and dict_loc for LSI and TF-IDF 

In [10]:
scored_list = scoring_module(name='cooccur', model_loc='/content/drive/My Drive/Movielensdata/ml25m/cooccur/co_occur', data=scoring_input, dict_loc='/content/drive/My Drive/Movielensdata/ml25m/tfidf/dict', matrix_location ='/content/drive/My Drive/Movielensdata/ml25m/cooccur/matrix' )

In [11]:
print(scored_list[0])

[30, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [12]:
#print(scored_list)

In [13]:
!pip install sentencepiece
!git clone https://github.com/google-research/bert

import os
import sys
import json
import nltk
import random
import logging
import tensorflow as tf
import tensorboard as tb
tf.gfile = tf.io.gfile
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile
import sentencepiece as spm

from glob import glob
from google.colab import auth, drive
from tensorflow.keras.utils import Progbar

sys.path.append("bert")

from bert import modeling, tokenization
#from bert.run_pretraining import input_fn_builder, model_fn_builder

     |████████████████████████████████| 1.1MB 2.9MB/s 
Cloning into 'bert'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 317.20 KiB | 8.35 MiB/s, done.
Resolving deltas: 100% (185/185), done.


In [14]:
bert_vocab = load_object('/content/drive/My Drive/Movielensdata/ml25m/cooccur/vocab')

In [15]:
print(bert_vocab)

['0', '1', '102407', '102686', '102716', '103141', '103384', '1036', '103810', '104241', '105213', '106489', '107348', '1080', '1088', '109578', '111113', '111443', '111659', '112138', '112171', '1136', '1175', '1196', '1197', '1198', '1200', '1201', '1210', '1214', '1217', '1220', '122896', '122906', '1237', '1246', '1250', '1257', '1260', '1265', '1270', '1271', '1275', '1278', '1287', '1288', '1291', '1293', '129354', '1302', '130634', '135143', '135536', '135569', '1356', '136020', '136864', '1376', '1393', '139644', '140956', '143385', '1465', '149406', '1527', '1580', '158238', '1584', '158966', '159093', '1610', '1653', '166635', '1672', '1674', '168612', '1693', '171763', '1732', '176101', '176371', '179819', '1873', '187541', '187593', '1907', '1923', '1960', '2011', '2012', '2019', '2028', '2068', '2083', '2115', '2138', '2139', '2150', '2161', '2268', '2294', '2324', '2351', '2355', '2359', '2501', '2571', '2573', '260', '261', '2632', '2692', '2716', '27193', '27266', '2745

In [16]:
ctrl_symbols = ["[PAD]","[UNK]","[CLS]","[SEP]","[MASK]"]
bert_vocab = ctrl_symbols + bert_vocab

In [17]:
print(bert_vocab)

['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]', '0', '1', '102407', '102686', '102716', '103141', '103384', '1036', '103810', '104241', '105213', '106489', '107348', '1080', '1088', '109578', '111113', '111443', '111659', '112138', '112171', '1136', '1175', '1196', '1197', '1198', '1200', '1201', '1210', '1214', '1217', '1220', '122896', '122906', '1237', '1246', '1250', '1257', '1260', '1265', '1270', '1271', '1275', '1278', '1287', '1288', '1291', '1293', '129354', '1302', '130634', '135143', '135536', '135569', '1356', '136020', '136864', '1376', '1393', '139644', '140956', '143385', '1465', '149406', '1527', '1580', '158238', '1584', '158966', '159093', '1610', '1653', '166635', '1672', '1674', '168612', '1693', '171763', '1732', '176101', '176371', '179819', '1873', '187541', '187593', '1907', '1923', '1960', '2011', '2012', '2019', '2028', '2068', '2083', '2115', '2138', '2139', '2150', '2161', '2268', '2294', '2324', '2351', '2355', '2359', '2501', '2571', '2573', '260', '261', '

In [18]:
VOC_FNAME = "vocab.txt"

with open(VOC_FNAME, "w") as fo:
  for token in bert_vocab:
    fo.write(token+"\n")

In [19]:
testcase = "'5952', '2012', '0'"

In [20]:
bert_tokenizer = tokenization.FullTokenizer(VOC_FNAME)
bert_tokenizer.tokenize(testcase)

['[UNK]',
 '5952',
 '[UNK]',
 '[UNK]',
 '[UNK]',
 '2012',
 '[UNK]',
 '[UNK]',
 '[UNK]',
 '0',
 '[UNK]']

In [21]:
MAX_SEQ_LENGTH = 128 #@param {type:"integer"}
MASKED_LM_PROB = 0.15 #@param
MAX_PREDICTIONS = 20 #@param {type:"integer"}
DO_LOWER_CASE = True #@param {type:"boolean"}
PROCESSES = 2 #@param {type:"integer"}
PRETRAINING_DIR = "pretraining_data" #@param {type:"string"}

In [22]:
tf.io.gfile.makedirs(PRETRAINING_DIR)

In [23]:
MODEL_DIR = "bert_model" #@param {type:"string"}
tf.io.gfile.makedirs(MODEL_DIR)

In [25]:
# use this for BERT-base
VOC_SIZE = 300
bert_base_config = {
  "attention_probs_dropout_prob": 0.1, 
  "directionality": "bidi", 
  "hidden_act": "gelu", 
  "hidden_dropout_prob": 0.1, 
  "hidden_size": 768, 
  "initializer_range": 0.02, 
  "intermediate_size": 3072, 
  "max_position_embeddings": 512, 
  "num_attention_heads": 12, 
  "num_hidden_layers": 12, 
  "pooler_fc_size": 768, 
  "pooler_num_attention_heads": 12, 
  "pooler_num_fc_layers": 3, 
  "pooler_size_per_head": 128, 
  "pooler_type": "first_token_transform", 
  "type_vocab_size": 2, 
  "vocab_size": VOC_SIZE
}

with open("{}/bert_config.json".format(MODEL_DIR), "w") as fo:
  json.dump(bert_base_config, fo, indent=2)
  
with open("{}/{}".format(MODEL_DIR, VOC_FNAME), "w") as fo:
  for token in bert_vocab:
    fo.write(token+"\n")